Prueba concepto

1. Importamos las librerías necesarias

In [1]:
import numpy as np
import random
import math
import openpyxl
import torch
from torch.nn import Linear
from torch.nn import Flatten
from torch.nn import MSELoss

2. importamos fichero de datos

In [2]:
path = "z:/TFM/vregresionkiaversion_stuttgart.xlsx"

In [3]:
vfichero=openpyxl.load_workbook(path, read_only=False,keep_vba=False, data_only=True, keep_links=False)

In [4]:
hojavdatos=vfichero["vdatos"]#


In [5]:
km=[]
litros=[]
kwh=[]
temp=[]


Obtenemos los datos de las hojas de excel

In [6]:
n=2 # primera fila a leer
while hojavdatos.cell(row=n,column=1).value!=None:# leemos las ciudades, son todos los mismos para cada hoja
    km.append(hojavdatos.cell(row=n,column=2).value)
    litros.append(hojavdatos.cell(row=n,column=4).value)
    kwh.append(hojavdatos.cell(row=n,column=7).value)
    temp.append(hojavdatos.cell(row=n,column=12).value)
    n+=1

metemos los datos en tensores

In [7]:
vlongitud=len(km)
litros_np=np.array(litros)
litros_np.reshape(vlongitud,1)
kwh_np=np.array(kwh)
kwh_np.reshape(vlongitud,1)
temp_np=np.array(temp)
temp_np.reshape(vlongitud,1)
x=np.empty(shape=(vlongitud, 3),dtype=float)
x[:,0]=litros_np
x[:,1]=kwh_np
x[:,2]=temp_np


In [14]:
km_t=torch.tensor(km)
x_t=torch.tensor(x,dtype=torch.float32)# para aplicar el linear
x_t.type()

'torch.FloatTensor'

In [20]:


lr = Linear(in_features=3, out_features=1, bias=True)
yhat = lr(x_t)
print("yhat:=",yhat)
print("yhatw:=",lr.weight)
model = torch.nn.Sequential(lr,
    torch.nn.Flatten(0, 1)
)
list(model.parameters())
print("model= ",model)
loss_fn = MSELoss(reduction='sum')
print("loss_fn= ",loss_fn)
print("The prediction: ", yhat)
print("Parameters w and b: ", list(lr.parameters()))


yhat:= tensor([[ 1.8986e-01],
        [ 1.8988e+00],
        [-4.2871e-01],
        [-3.0435e-01],
        [ 8.0536e-01],
        [ 1.2767e-01],
        [ 8.8834e-02],
        [ 8.7521e-01],
        [ 1.0350e+00],
        [ 5.7225e-02],
        [ 2.4459e+00],
        [-8.7347e-01],
        [-7.9236e-01],
        [ 1.1241e-01],
        [ 1.1208e+00],
        [-4.8138e-01],
        [-4.8333e-01],
        [-1.2264e-01],
        [-9.3314e-02],
        [-1.9880e-01],
        [-2.1124e-01],
        [-1.7556e-01],
        [-1.8887e-02],
        [ 2.4202e-02],
        [ 3.6641e-02],
        [-1.8906e-01],
        [ 4.4682e-01],
        [-2.5459e-01],
        [-7.1644e-01],
        [-9.5585e-01],
        [-3.1718e-01],
        [-3.1896e-01],
        [ 7.7104e-01],
        [-5.7750e-01],
        [ 3.7735e-02],
        [-5.6398e-01],
        [-6.1070e-01],
        [-5.3737e-01],
        [-5.8534e-01],
        [ 2.9045e-01],
        [ 4.4767e-02],
        [-7.0913e-01],
        [-7.3756e-01],
    

In [24]:
learning_rate = 1e-6
for t in range(4000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_t)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, km_t.T)
    print("error:= ",loss)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    #optimizer.step()
    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            print("param.grad:",param.grad)
# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: km = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} litros + {linear_layer.weight[:, 1].item()} kwh + {linear_layer.weight[:, 2].item()} temp')
#print('Weight Of Network :\n',m.weight)


error:=  tensor(10019257., grad_fn=<MseLossBackward0>)
param.grad: tensor([[-1337561.5000,  -348492.5312, -1078556.5000]])
param.grad: tensor([-67867.0547])
error:=  tensor(7332945.5000, grad_fn=<MseLossBackward0>)
param.grad: tensor([[-1155471.2500,  -242012.8125,  -613294.8125]])
param.grad: tensor([-42564.3828])
error:=  tensor(5739322.5000, grad_fn=<MseLossBackward0>)
param.grad: tensor([[-1014140.3125,  -174175.8438,  -326954.7812]])
param.grad: tensor([-26727.5488])
error:=  tensor(4662904., grad_fn=<MseLossBackward0>)
param.grad: tensor([[-900679.2500, -130637.9453, -152276.1250]])
param.grad: tensor([-16819.3477])
error:=  tensor(3863314.7500, grad_fn=<MseLossBackward0>)
param.grad: tensor([[-806813.5625, -102404.8594,  -47155.9727]])
param.grad: tensor([-10623.7744])
error:=  tensor(3234101., grad_fn=<MseLossBackward0>)
param.grad: tensor([[-727175.7500,  -83834.3438,   14751.4043]])
param.grad: tensor([-6752.5542])
error:=  tensor(2723361., grad_fn=<MseLossBackward0>)
param.g

error:=  tensor(140378.7500, grad_fn=<MseLossBackward0>)
99 140378.75
param.grad: tensor([[   797.8235, -12868.8838,   2553.8247]])
param.grad: tensor([684.3386])
error:=  tensor(140206.3281, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   802.1709, -12748.8643,   2526.9634]])
param.grad: tensor([694.3960])
error:=  tensor(140037.0156, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   805.5477, -12630.0078,   2500.6482]])
param.grad: tensor([704.3611])
error:=  tensor(139870.9219, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   807.9056, -12512.3184,   2474.6526]])
param.grad: tensor([714.2256])
error:=  tensor(139707.7969, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   809.4788, -12395.7471,   2449.1162]])
param.grad: tensor([723.9981])
error:=  tensor(139547.7344, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   810.1147, -12280.3184,   2423.9429]])
param.grad: tensor([733.6746])
error:=  tensor(139390.5625, grad_fn=<MseLossBackward0>)
param.grad: tensor([[   810.13

error:=  tensor(132152.5625, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  338.6884, -4693.1548,   864.5469]])
param.grad: tensor([1359.2692])
error:=  tensor(132127.9062, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  335.4492, -4650.5083,   855.8672]])
param.grad: tensor([1362.6509])
error:=  tensor(132103.6875, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  332.2273, -4608.2554,   847.2363]])
param.grad: tensor([1365.9983])
error:=  tensor(132079.8906, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  329.1384, -4566.3931,   838.7588]])
param.grad: tensor([1369.3132])
error:=  tensor(132056.4219, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  325.9746, -4524.9443,   830.2874]])
param.grad: tensor([1372.5911])
error:=  tensor(132033.3750, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  322.9244, -4483.8750,   821.9814]])
param.grad: tensor([1375.8373])
error:=  tensor(132010.6875, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  319.8103, -4443.1953,   813.6699

param.grad: tensor([[  113.4834, -1701.8805,   258.0298]])
param.grad: tensor([1582.3948])
error:=  tensor(130784.4531, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  112.3593, -1687.3503,   255.0657]])
param.grad: tensor([1583.3347])
error:=  tensor(130779.0781, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  111.3317, -1672.9603,   252.1331]])
param.grad: tensor([1584.2632])
error:=  tensor(130773.7109, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  110.1957, -1658.7097,   249.2144]])
param.grad: tensor([1585.1787])
error:=  tensor(130768.3438, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  109.0505, -1644.5833,   246.3682]])
param.grad: tensor([1586.0850])
error:=  tensor(130763.0703, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  108.1021, -1630.5747,   243.5852]])
param.grad: tensor([1586.9822])
error:=  tensor(130757.8125, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  107.0385, -1616.7012,   240.8069]])
param.grad: tensor([1587.8672])
error:=  tensor(13075

param.grad: tensor([[  37.1525, -686.8773,   53.4263]])
param.grad: tensor([1634.4957])
error:=  tensor(130340.2266, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  36.7683, -681.8744,   52.3835]])
param.grad: tensor([1634.6125])
error:=  tensor(130337.0703, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  36.3485, -676.9066,   51.3816]])
param.grad: tensor([1634.7263])
error:=  tensor(130333.9141, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  35.9979, -671.9846,   50.4204]])
param.grad: tensor([1634.8389])
error:=  tensor(130330.8125, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  35.5696, -667.1206,   49.4260]])
param.grad: tensor([1634.9438])
error:=  tensor(130327.6953, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  35.3143, -662.2767,   48.5273]])
param.grad: tensor([1635.0504])
error:=  tensor(130324.5469, grad_fn=<MseLossBackward0>)
param.grad: tensor([[  34.9622, -657.5055,   47.5613]])
param.grad: tensor([1635.1475])
error:=  tensor(130321.4453, grad_fn=<MseL

...

param.grad: tensor([[ -1.4956, -89.6149, -30.8376]])
param.grad: tensor([955.2255])
error:=  tensor(125181.1562, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4374, -89.5889, -30.8035]])
param.grad: tensor([955.0485])
error:=  tensor(125180.2031, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.3588, -89.5712, -30.7810]])
param.grad: tensor([954.8701])
error:=  tensor(125179.3359, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4165, -89.5605, -30.8098]])
param.grad: tensor([954.6892])
error:=  tensor(125178.3750, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4886, -89.5488, -30.8223]])
param.grad: tensor([954.5087])
error:=  tensor(125177.4531, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4746, -89.5280, -30.8018]])
param.grad: tensor([954.3305])
error:=  tensor(125176.5469, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4180, -89.4968, -30.7390]])
param.grad: tensor([954.1545])
error:=  tensor(125175.6406, grad_fn=<MseLossBackward0>)
param.grad: t

param.grad: tensor([935.3606])
error:=  tensor(125080.1016, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.5009, -87.7467, -30.2283]])
param.grad: tensor([935.1839])
error:=  tensor(125079.2344, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4285, -87.7265, -30.1833]])
param.grad: tensor([935.0101])
error:=  tensor(125078.3516, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4639, -87.7116, -30.1948]])
param.grad: tensor([934.8334])
error:=  tensor(125077.4219, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4957, -87.6807, -30.1567]])
param.grad: tensor([934.6598])
error:=  tensor(125076.5547, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4099, -87.6633, -30.1365]])
param.grad: tensor([934.4863])
error:=  tensor(125075.7031, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4659, -87.6590, -30.1648]])
param.grad: tensor([934.3086])
error:=  tensor(125074.8359, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4559, -87.6274, -30.1396]])
param.grad: t

param.grad: tensor([915.7361])
error:=  tensor(124982.3984, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.3979, -85.8982, -29.5549]])
param.grad: tensor([915.5669])
error:=  tensor(124981.5469, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4834, -85.8881, -29.5732]])
param.grad: tensor([915.3932])
error:=  tensor(124980.7188, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4213, -85.8719, -29.5518]])
param.grad: tensor([915.2231])
error:=  tensor(124979.8516, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.5162, -85.8622, -29.5684]])
param.grad: tensor([915.0499])
error:=  tensor(124979.0156, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4202, -85.8338, -29.5044]])
param.grad: tensor([914.8809])
error:=  tensor(124978.1641, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.5120, -85.8275, -29.5300]])
param.grad: tensor([914.7068])
error:=  tensor(124977.2969, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4380, -85.7971, -29.5020]])
param.grad: t

error:=  tensor(124891.9609, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4716, -84.1420, -28.9583]])
param.grad: tensor([897.0336])
error:=  tensor(124891.1719, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4065, -84.1236, -28.9424]])
param.grad: tensor([896.8658])
error:=  tensor(124890.3438, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4507, -84.1228, -28.9707]])
param.grad: tensor([896.6946])
error:=  tensor(124889.4922, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4408, -84.0988, -28.9512]])
param.grad: tensor([896.5271])
error:=  tensor(124888.7344, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.3739, -84.0831, -28.9370]])
param.grad: tensor([896.3600])
error:=  tensor(124887.9219, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4091, -84.0824, -28.9666]])
param.grad: tensor([896.1893])
error:=  tensor(124887.1172, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4747, -84.0699, -28.9666]])
param.grad: tensor([896.0205])
error:=  tens

param.grad: tensor([[ -1.3998, -82.4242, -28.3535]])
param.grad: tensor([878.5432])
error:=  tensor(124802.8203, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4875, -82.4091, -28.3584]])
param.grad: tensor([878.3777])
error:=  tensor(124802.0938, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.3937, -82.3853, -28.3005]])
param.grad: tensor([878.2158])
error:=  tensor(124801.2812, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.5076, -82.3735, -28.3335]])
param.grad: tensor([878.0486])
error:=  tensor(124800.5156, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4187, -82.3520, -28.3130]])
param.grad: tensor([877.8864])
error:=  tensor(124799.6953, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4935, -82.3394, -28.3062]])
param.grad: tensor([877.7207])
error:=  tensor(124798.9609, grad_fn=<MseLossBackward0>)
param.grad: tensor([[ -1.4385, -82.3145, -28.2939]])
param.grad: tensor([877.5574])
error:=  tensor(124798.1562, grad_fn=<MseLossBackward0>)
3899 124798.1

In [25]:
print("loss",loss)

loss tensor(124721.8594, grad_fn=<MseLossBackward0>)


3. Cargamos la poblacion.

seleccionados aquellas distancia que sean menor de 550 minutos en total con una busqueda alatoria. 

hacemos solo 20 busquedas aleatorias

In [10]:
vdistanciatotal.shape # distancias totales de las busquedas
    
    
    


(680,)

In [11]:
vdistanciatotal



array([488, 390, 249, 353, 160, 398, 345, 380, 455, 352, 213, 368, 271,
       525, 390, 197, 280, 412, 483, 543, 295, 335, 221, 366, 343, 491,
       352, 529, 411, 306, 333, 510, 411, 410, 401, 241, 222, 290, 283,
       283, 307, 289, 528, 431, 251, 361, 272, 323, 270, 204, 332, 442,
       543, 543, 298, 318, 176, 219, 287, 414, 336, 503, 429, 325, 297,
       426, 477, 232, 439, 408, 199, 294, 165, 298, 243, 397, 444, 349,
       182, 314, 296, 367, 336, 160, 197, 368, 528, 540, 344, 243, 235,
       367, 261, 367, 429, 534, 493, 189, 440, 381, 323, 352, 378, 276,
       285, 290, 254, 336, 233, 550, 459, 302, 146, 376, 319, 369, 260,
       256, 356, 387, 523, 530, 341, 311, 187, 342, 246, 360, 369, 507,
       418, 253, 256, 425, 343, 276, 364, 427, 206, 299, 226, 290, 316,
       493, 289, 296, 257, 266, 229, 382, 385, 175, 326, 447, 549, 535,
       199, 257, 167, 356, 239, 374, 265, 528, 441, 154, 349, 464, 391,
       438, 348, 431, 160, 374, 222, 365, 383, 321, 314, 420, 24

In [12]:
vcentrossseleccion

array([['61352 BAD HOMBURG ', '61352 BAD HOMBURG ', '64285 DARMSTADT ',
        '35578 WETZLAR', '63450 HANAU ', '64823 GROß-UMSTADT '],
       ['64285 DARMSTADT ', '64285 DARMSTADT ',
        '60438 FRANKFURT (Riedberg) ',
        '65929 FRANKFURT (Frankfurt-Höchs', '50672 KÖLN ',
        '60594 FRANKFURT (Frankfurt -Südb'],
       ['60326 FRANKFURT (Frankfurt - Gal',
        '60326 FRANKFURT (Frankfurt - Gal', '65779 KELKHEIM ',
        '64646 HEPPENHEIM ', '63225 LANGEN ', '65428 RÜSSELSHEIM '],
       ...,
       ['64823 GROß-UMSTADT ', '65795 HATTERSHEIM ', '63071 OFFENBACH  ',
        '34119 KASSEL ', '64319 PFUNGSTADT  ', '64823 GROß-UMSTADT '],
       ['65428 RÜSSELSHEIM ', '60313 FRANKFURT (Frankfurt – Wes',
        '60486 FRANKFURT (Bockenheim)  ', '35578 WETZLAR',
        '63450 HANAU ', '65428 RÜSSELSHEIM '],
       ['65185 WIESBADEN Oranienschule', '65185 WIESBADEN Oranienschule',
        '60486 FRANKFURT (Bockenheim)  ', '63457 HANAU (Großauheim) ',
        '64319 PFUNGST

volcamos los resultados posibles en la hoja excel para visualizarlos
    

In [13]:
hojaresultados=vfichero["resultados"]
#hojaresultados= vfichero.get_sheet_by_name('resultados')
for i4 in range(len(vcentrossseleccion)):
    for i5 in range(len(vcentrossseleccion[0])):
        hojaresultados.cell(row=i4+2, column=i5+1,value= vcentrossseleccion[i4,i5])
    for i6 in range(len(vdistanciasseleccion[0])):
        hojaresultados.cell(row=i4+2, column=i6+9,value= vdistanciasseleccion[i4,i6])    
        hojaresultados.cell(row=i4+2, column=i6+11,value= np.transpose(vdistanciatotal[i4]))  
       





        
        
               


In [14]:
vdistanciasseleccion


array([[  0, 149, 130,  95, 114],
       [  0, 122,  62, 150,  56],
       [  0,  63,  62,  50,  74],
       ...,
       [122,  66, 183,  97,   0],
       [ 71,  74, 169, 119,   0],
       [  0,  72, 100, 107,   0]])

vamos a seleccionar 6 profesores los cuales tiene que dar todos los dias clase

In [15]:
vcentros,vdist=mutacion.selcentromartes("martes",centrolunes,centromartes,centromiercoles,centrojueves,centroviernes,vcentrossseleccion,vdistanciatotal)


list-lunes= [0, 1, 2, 3, 4, 5, 6, 7]
depues_list_lunes= [1, 2, 3, 4, 5, 6, 7]
mi= 6
centros= ['61352 BAD HOMBURG ' '61352 BAD HOMBURG ' '64285 DARMSTADT '
 '35578 WETZLAR' '63450 HANAU ' '64823 GROß-UMSTADT ']
a2= 0 a3= 1 lu= 7 ma= 5 mi= 6 ju= 6 vi= 5
list-lunes= [1, 2, 3, 4, 5, 6, 7]
depues_list_lunes= [1, 2, 3, 4, 5, 7]
mi= 5
centros= ['63179 OBERTSHAUSEN ' '63179 OBERTSHAUSEN '
 '60486 FRANKFURT (Bockenheim)  ' '34119 KASSEL ' '65812 BAD SODEN '
 '60438 FRANKFURT (Riedberg) ']
a2= 6 a3= 2 lu= 6 ma= 4 mi= 5 ju= 5 vi= 4
list-lunes= [1, 2, 3, 4, 5, 7]
depues_list_lunes= [2, 3, 4, 5, 7]
mi= 4
centros= ['65185 WIESBADEN Jahnschule ' '64285 DARMSTADT '
 '65185 WIESBADEN Oranienschule' '65185 WIESBADEN Jahnschule '
 '64319 PFUNGSTADT  ' '65428 RÜSSELSHEIM ']
a2= 54 a3= 3 lu= 5 ma= 3 mi= 4 ju= 4 vi= 3
list-lunes= [2, 3, 4, 5, 7]
depues_list_lunes= [2, 3, 4, 7]
mi= 3
centros= ['63225 LANGEN ' '65795 HATTERSHEIM ' '60438 FRANKFURT (Riedberg) '
 '61118 BAD VILBEL ' '63225 LANGEN ' '65185 WIESB

imprimimos en excel los resultados finales

In [16]:
i4=0
i5=0
i6=0
i7=0
a=0
hojaresultados2=vfichero["final"]
#hojaresultados= vfichero.get_sheet_by_name('resultados')
for i4 in range(len(vcentros)):
    vresultados.cell(row=i4+2, column=9,value= vdist[i4,0])   
   
    for i5 in range(len(vcentros[0])):
            hojaresultados2.cell(row=i4+2, column=i5+1,value= vcentros[i4,i5])
    
         

         
        
        
vfichero.save(filename = path)